# Hmac python



### Challenge Name: hmac_generate (/embsec/hmac_python/hmac_generate)


    The serial device will send you a variable-length frame of data. The data
    frame will begin with a short, little-endian integer size of the data to 
    follow. You must generate an HMAC-SHA256 signature using the key in 
    'hmackeyfile0.bin'.
    
    The data frame you will receive from the device will be formatted as follows:
    
    [ 0x2   ] [   variable...   ]
    ------------------------------
    | Size   |     Data          |
    ------------------------------
    
    You must send a 32-byte HMAC-SHA256 signature to the serial device.
    
    [     0x20     ]
    ----------------
    |  HMAC(Data)  |
    ----------------
    
    1. Read the HMAC key from 'hmackeyfile0.bin'
    2. Read the size of the data from the serial device
    3. Read the data from the serial device
    4. Generate and send a HMAC-SHA256 over the data
    5. Read the response
    



In [1]:
from embsec import Serial
import struct
from Crypto.Hash import HMAC, SHA256

def hmac_generate():
    ser = Serial("/embsec/hmac_python/hmac_generate")
    # Your code goes here!
    
    with open("hmackeyfile0.bin", "rb") as f:
        key0 = f.read()
    data_size = struct.unpack('<h', ser.read(2))[0]
    data = struct.unpack('<{}s'.format(data_size), ser.read(data_size))[0]

    
    h = HMAC.new(key0, digestmod=SHA256)
    new_hmac = h.update(data)
    
    ser.write(new_hmac.digest())
    
    return ser.read_until()
hmac_generate()


b'embsec{hmac_generate_40a7c2dbd2fcb709}\n'

In [2]:
from Crypto.Hash import HMAC, SHA256
secret = b'Swordfish'
h = HMAC.new(secret, digestmod=SHA256)
print(h.hexdigest())
h.update(b'Hello')
print(h.digest())
print(h.hexdigest())

c3f33e3055bdc7534ba7d6d7c941dc0a039169510af95674f84f195759744205
b"\x11A\xda\xfd\x14F-\xc3\x9a>\x11\x07\xe4\x14\x12\xb3\xad'\x18Ify\xae:\x0fp\x14\x07\xdc\x87_I"
1141dafd14462dc39a3e1107e41412b3ad2718496679ae3a0f701407dc875f49


### Challenge Name: hmac_verify (/embsec/hmac_python/hmac_verify)


    The serial device will send you a series of messages in the format
    described below. For each message you must check the attached signature 
    If verification fails, you must respond with a zero-byte '\x00'. If 
    verification passes, you must respond with a one-byte '\x01'. When the
    length of the message you are about to receive is zero , read a newline-
    terminated flag.

    The serial device will send signed messages in the following format:

    ----------------------------------------
    [ 0x2 ] [  Size bytes    ] [   0x20   ]
    ----------------------------------------
    | Size |     Data         | HMAC(Data) |
    ----------------------------------------



In [4]:
from embsec import Serial
import struct
from Crypto.Hash import HMAC, SHA256

def hmac_verify():
    ser = Serial("/embsec/hmac_python/hmac_verify")
    # Your code goes here!
    with open("hmackeyfile1.bin", "rb") as f:
        key1 = f.read()
    
   
    data_size = struct.unpack('<h', ser.read(2))[0]
    while (data_size):

        data = ser.read(data_size)
        given_hmac = ser.read(32)
        h = HMAC.new(key1, digestmod=SHA256)
        h.update(data)
        
   
        try:
            h.verify(given_hmac)
        except ValueError:
            ser.write(b'\x00')
            print("0")
        else:
            ser.write(b'\x01')
            print("1")
        
        data_size = struct.unpack('<h', ser.read(2))[0]
        

    return ser.read_until().decode()

hmac_verify()


0
0
1
0
1
1
1
0


'embsec{hmac_verify_85eecbb6339cc72c}\n'